In [1]:
import os
import pandas as pd
import gzip
import json
import numpy as np
import time

In [ ]:
data_0 = pd.read_csv('all_result/reference_full_information_final_v0.csv', dtype = str)

## get isbn list of library book collection data

In [ ]:
file_print = open('uwmsn-greenglass-books/books-print.mrk',encoding='utf-8')
print_text = file_print.readlines()

In [ ]:
print_book_dict = {}
for line_0 in print_text:
    line = line_0.replace('-','')
    if line.startswith('=020  \\\\$a'):
        if len(line)<21:
            continue
        if line[20].isdigit():  #check the isbn is new version (13 digits) or old version (10 digits)
            print_book_dict[line[10:23]]=line
        else:
            print_book_dict[line[10:20]]=line

In [ ]:
file_print.close()

In [ ]:
file_eprint = open('uwmsn-greenglass-books/books-electronic.mrk',encoding='utf-8')
eprint_text = file_eprint.readlines()

In [ ]:
eprint_book_dict = {}
eprint_list = []
for line_0 in eprint_text:
    line = line_0.replace('-','')
    if line.startswith('=020  \\\\$a'):
        eprint_list.append(line)
        if len(line)<21:
            continue
        if line[20].isdigit():
            eprint_book_dict[line[10:23]]=line
        else:
            eprint_book_dict[line[10:20]]=line

## get isbn list of library venue(journal/conference) collection data

In [ ]:
file_venue = pd.read_csv("Query Example_ Find by Bib Material Type.csv")

In [ ]:
issn_list_0 = file_venue[~file_venue['ISSN'].isnull()]['ISSN'].to_list()
issn_dict = {}
for x in issn_list_0:
    if ';' in x:
        x_list = x.split("; ")
        for m in x_list:
            if m not in issn_dict:
                issn_dict[m]=[]
    else:
        if x not in issn_dict:
            issn_dict[x]=[]

## match issn and isbn

### match issn

In [ ]:
match_issn = []
data_wos_issn = data_0['issn'].to_list()
data_wos_eissn = data_0['eissn'].to_list()
data_wos_issn2 = data_0['issn_2'].to_list() # In Wos, some publications may have 2 issn in its record

In [ ]:
for i in range(len(data_wos_issn)):
    match_issn.append(0)
    if data_wos_issn[i] in issn_dict:
        match_issn[i] = match_issn[i]+1
        continue
    if data_wos_eissn[i] in issn_dict:
        match_issn[i] = match_issn[i]+1
        continue
    if data_wos_issn2[i] in issn_dict:
        match_issn[i] = match_issn[i]+1
        continue
    

In [ ]:
data_0['match_issn'] = np.array(match_issn)

In [ ]:
match_isbn = []
data_wos_isbn = data_0['isbn'].to_list()
data_wos_eisbn = data_0['eisbn'].to_list()

In [ ]:
m=0
n=0
mn = 0
for i in range(len(data_wos_isbn)):
    match_isbn.append(0)
    if pd.isna(data_wos_isbn[i])==False:
        m=m+1
        if (data_wos_isbn[i] in print_book_dict)|(data_wos_isbn[i].replace('-','') in print_book_dict):
            match_isbn[i] = match_isbn[i]+1
            mn = mn+1
            continue
        if (data_wos_isbn[i] in eprint_book_dict)|(data_wos_isbn[i].replace('-','') in eprint_book_dict):
            match_isbn[i] = match_isbn[i]+1
            mn = mn+1
            continue
    if pd.isna(data_wos_eisbn[i])==False:
        n=n+1
        if (data_wos_eisbn[i] in print_book_dict)|(data_wos_eisbn[i].replace('-','') in print_book_dict):
            match_isbn[i] = match_isbn[i]+1
            mn = mn+1
            continue
        if (data_wos_eisbn[i] in eprint_book_dict)|(data_wos_eisbn[i].replace('-','') in eprint_book_dict):
            match_isbn[i] = match_isbn[i]+1
            mn = mn+1
            continue

In [ ]:
data_0['match_isbn'] = np.array(match_isbn)

In [ ]:
data_0.to_csv('all_result/reference_full_information_v4.csv',index=False)

In [ ]:
data_0

,id,2018,2019,2020,2021,2022,year,author,title,work,...,eissn,publication_type,heading,heading_len,isbn,eisbn,doc_type,issn_2,match_issn,match_isbn
0,WOS:000218666500001,1,0,0,0,0,2013,"Poullikkas, A",A comparative assessment of net metering and f...,SUSTAINABLE ENERGY TECHNOLOGIES AND ASSESSMENTS,...,2213-1396,Journal,['Science & Technology'],1.0,NaN,NaN,['Article'],NaN,1,0
1,000444741500122.11,1,0,0,0,0,2017,"Tran, V. T.",A robust power management strategy entrenched ...,P IEEE IND APPL SOC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,WOS:000331985300047,1,0,0,0,0,2014,"Yang, Y",Sizing Strategy of Distributed Battery Storage...,IEEE TRANSACTIONS ON SMART GRID,...,1949-3061,Journal,['Science & Technology'],1.0,NaN,NaN,['Article'],NaN,1,0
3,WOS:000354945400060,1,0,0,0,0,2015,"Kim, ST",Energy Management Based on the Photovoltaic HP...,IEEE TRANSACTIONS ON INDUSTRIAL ELECTRONICS,...,1557-9948,Journal,['Science & Technology'],1.0,NaN,NaN,['Article'],NaN,1,0
4,WOS:000426811700056,1,0,0,0,0,2017,"Prapanukool, C",An Appropriate Battery Capacity and Operation ...,2017 14TH INTERNATIONAL CONFERENCE ON ELECTRIC...,...,NaN,Book,['Science & Technology'],1.0,NaN,978-1-5386-0449-6,['Meeting'],NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1653697,000742181300001.10,0,0,0,0,1,2015,The American College of Obstetricians and Gyne...,NaN,Obesity in pregnancy. Practice Bulletin No. 15...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1653698,WOS:000302613800014,0,0,0,0,1,2012,"Bonner, JC",Respiratory arrest in an obstetric patient usi...,ANAESTHESIA,...,NaN,Journal,['Science & Technology'],1.0,NaN,NaN,['Article'],NaN,1,0
1653699,WOS:000314640800009,0,0,0,0,1,2013,"Marr, R",Cardiac arrest in an obstetric patient using r...,ANAESTHESIA,...,1365-2044,Journal,['Science & Technology'],1.0,NaN,NaN,['Article'],NaN,1,0
1653700,000742181300001.32,0,0,0,0,1,2005,The American College of Obstetricians and Gyne...,Pregestational diabetes mellitus. ACOG Practic...,Obstet Gynecol,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [ ]:
data_0['match_isbn'] = data_0['match_isbn'].astype('int16')
data_0['match_issn'] = data_0['match_issn'].astype('int16')
data_0['match_final'] = data_0['match_isbn']+data_0['match_issn']
match_final_list = data_0['match_final'].to_list()
match_final_list_2 = [1 if x==2 else x for x in match_final_list]
data_0['match_final']=match_final_list_2

In [ ]:
data_0.groupby(['publication_type','match_final']).count()

id     2018     2019     2020     2021  \
publication_type match_final                                                
Book             0               6841     6841     6841     6841     6841   
                 1               3082     3082     3082     3082     3082   
Book in series   0               5358     5358     5358     5358     5358   
                 1              21265    21265    21265    21265    21265   
Journal          0              22944    22944    22944    22944    22944   
                 1            1160331  1160331  1160331  1160331  1160331   

                                 2022     year   author    title     work  \
publication_type match_final                                                
Book             0               6841     6841     6696     6841     6841   
                 1               3082     3082     2932     3082     3082   
Book in series   0               5358     5358     5228     5358     5358   
                 1              21265    21265    21126    21265    21265   
Journal          0              22944    22944    22932    22944    22944   
                 1            1160331  1160331  1160090  1160327  1160331   

                              ...     issn   eissn  heading  heading_len  \
publication_type match_final  ...                                          
Book             0            ...        0       0     6837         6841   
                 1            ...        0       0     3078         3082   
Book in series   0            ...     3544     530     5356         5358   
                 1            ...    19423    6089    21255        21265   
Journal          0            ...    20333   14707    22931        22944   
                 1            ...  1134186  753892  1160127      1160331   

                               isbn  eisbn  doc_type  issn_2  match_issn  \
publication_type match_final                                               
Book             0             2523   4967      6841       0        6841   
                 1             2303   2307      3082       0        3082   
Book in series   0             2301   4034      5358       0        5358   
                 1            11367  14047     21265       0       21265   
Journal          0                0      0     22944       0       22944   
                 1               68      0   1160331       0     1160331   

                              match_isbn  
publication_type match_final              
Book             0                  6841  
                 1                  3082  
Book in series   0                  5358  
                 1                 21265  
Journal          0                 22944  
                 1               1160331  

[6 rows x 25 columns]